# 🚀 SwingAI Training V4 — Colab (Optimized + Tracked)

This notebook is a **cleanly split** version of your `SwingAI_Colab_V4_Fixed.py`, with:

- ✅ Proper notebook sections (each `# CELL X:` becomes its own cell)
- ⚡ Colab T4 optimizations (AMP mixed precision, dataloaders tuned, safer memory patterns)
- 📈 Experiment tracking (run folder, config snapshot, CSV metrics, TensorBoard logs, checkpoints)

> Tip: If you want logs & checkpoints persisted, mount Google Drive in the next cell.


In [1]:
# (Optional) Mount Google Drive for persistent runs/checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Choose where runs are stored (Drive recommended). You can change this.
RUNS_BASE_DIR = "/content/drive/MyDrive/SwingAI_runs"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ✅ Install (run once, then Runtime → Restart runtime)
# Note: Colab already has PyTorch + CUDA. We pin versions that commonly break projects.
%pip -q install --upgrade "pip<25"
%pip -q install "numpy<2.0" "pandas==2.2.2" "scipy<1.13" "scikit-learn==1.5.2" \
    "catboost==1.2.7" "yfinance>=0.2.54" "curl_cffi>=0.7.4" "tensorboard>=2.15"

print("✅ Install complete. Now restart the runtime.")


✅ Install complete. Now restart the runtime.


In [3]:
# 🔧 Runtime setup (speed + reproducibility + tracking)
import os, json, time, random, warnings
from datetime import datetime
from dataclasses import dataclass
from collections import Counter

import numpy as np
import pandas as pd
import yfinance as yf

from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.tensorboard import SummaryWriter

warnings.filterwarnings("ignore")

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False  # speed > determinism
    torch.backends.cudnn.benchmark = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   {torch.cuda.get_device_name(0)}")

# AMP mixed precision for T4 speed
USE_AMP = bool(torch.cuda.is_available())
SCALER = torch.cuda.amp.GradScaler(enabled=USE_AMP)

# Run folder (defaults to /content/SwingAI_runs unless you set RUNS_BASE_DIR earlier)
RUNS_BASE_DIR = globals().get("RUNS_BASE_DIR", "/content/SwingAI_runs")
os.makedirs(RUNS_BASE_DIR, exist_ok=True)

RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = os.path.join(RUNS_BASE_DIR, f"run_{RUN_ID}")
os.makedirs(RUN_DIR, exist_ok=True)

WRITER = SummaryWriter(log_dir=os.path.join(RUN_DIR, "tb"))

METRICS_CSV = os.path.join(RUN_DIR, "metrics.csv")
with open(METRICS_CSV, "w") as f:
    f.write("time,model,split,epoch,metric,value\n")

def log_metric(model: str, split: str, epoch: int, metric: str, value: float):
    t = datetime.now().isoformat(timespec="seconds")
    with open(METRICS_CSV, "a") as f:
        f.write(f"{t},{model},{split},{epoch},{metric},{value}\n")
    WRITER.add_scalar(f"{model}/{split}/{metric}", float(value), epoch)

print(f"📁 RUN_DIR: {RUN_DIR}")


✅ GPU available: True
   Tesla T4
📁 RUN_DIR: /content/drive/MyDrive/SwingAI_runs/run_20260107_182456


## Sections
- CELL 3: Config V4
- CELL 4: Download Data
- CELL 5: Features V4 (Less aggressive capping)
- CELL 6: Dataset (Standard - no balanced sampling for CatBoost)
- CELL 7: Soft Focal Loss (reduced gamma)
- CELL 8: CatBoost V4 - FIXED!
- CELL 9: TFT V4
- CELL 10: Stockformer V4
- CELL 11: Ensemble Evaluation
- CELL 12: Save Models

## CELL 3: Config V4

In [4]:
# CELL 3: Config V4
# ============================================================
@dataclass
class ConfigV4:
    LOOKBACK_DAYS: int = 60
    PREDICTION_HORIZON: int = 5
    MIN_HISTORY_DAYS: int = 252

    # Keep balanced thresholds (helped class balance)
    UP_THRESHOLD: float = 0.025
    DOWN_THRESHOLD: float = -0.015

    TRAIN_END: str = "2024-06-30"
    VAL_END: str = "2024-09-30"

    # Adjusted weights based on V2 performance
    CATBOOST_WEIGHT: float = 0.35
    TFT_WEIGHT: float = 0.40
    STOCKFORMER_WEIGHT: float = 0.25

    BATCH_SIZE: int = 64  # Back to V2 size
    LEARNING_RATE: float = 1e-3  # Back to V2
    MAX_EPOCHS: int = 50
    PATIENCE: int = 10

    # CatBoost V4 - FIXED
    CATBOOST_ITERATIONS: int = 1500
    CATBOOST_DEPTH: int = 7  # Slightly deeper than V2
    CATBOOST_LR: float = 0.04
    CATBOOST_L2_REG: float = 2.0

    # TFT - Between V2 and V3
    TFT_HIDDEN_SIZE: int = 96
    TFT_ATTENTION_HEADS: int = 6
    TFT_DROPOUT: float = 0.15
    TFT_NUM_LAYERS: int = 2

    # Stockformer - Between V2 and V3
    STOCKFORMER_D_MODEL: int = 96
    STOCKFORMER_N_HEADS: int = 6
    STOCKFORMER_N_LAYERS: int = 2
    STOCKFORMER_DROPOUT: float = 0.1

    # Loss - REDUCED focal gamma
    FOCAL_GAMMA: float = 1.0  # Reduced from 2.0
    LABEL_SMOOTHING: float = 0.05  # Reduced from 0.1

    NUM_FEATURES: int = 40
    MODEL_SAVE_PATH: str = "/content/drive/MyDrive/SwingAI/models_v4/"

config = ConfigV4()

FEATURE_NAMES = [
    'return_1d', 'return_5d', 'return_10d', 'return_20d', 'volatility_20d',
    'close_to_sma_20', 'close_to_sma_50', 'rsi_14_norm', 'macd_histogram_norm', 'bb_position',
    'structure_score', 'range_position', 'dist_to_swing_high', 'dist_to_swing_low',
    'in_discount', 'in_deep_discount', 'in_premium', 'near_bullish_ob', 'near_bearish_ob',
    'bullish_fvg', 'bearish_fvg', 'sweep_high', 'sweep_low', 'bos_bullish', 'bos_bearish',
    'volume_ratio', 'volume_trend', 'obv_slope', 'close_to_vwap',
    'buying_pressure', 'accumulation_score', 'big_volume_day', 'higher_high',
    'daily_trend', 'weekly_trend', 'monthly_trend', 'mtf_alignment',
    'weekly_range_pos', 'monthly_range_pos', 'trend_strength',
]

print(f"✅ Config V4: {len(FEATURE_NAMES)} features")

# ============================================================

# --- Colab/T4 extras ---
config.SEED = getattr(config, 'SEED', 42)
config.NUM_WORKERS = getattr(config, 'NUM_WORKERS', 2)
config.PIN_MEMORY = torch.cuda.is_available()
config.RUN_DIR = RUN_DIR

set_seed(config.SEED)

# Save config snapshot
with open(os.path.join(RUN_DIR, 'config_v4.json'), 'w') as f:
    json.dump(config.__dict__ if hasattr(config, '__dict__') else {}, f, indent=2, default=str)
print('✅ Saved config snapshot')


✅ Config V4: 40 features
✅ Saved config snapshot


## CELL 4: Download Data

In [5]:
# CELL 4: Download Data
# ============================================================
FO_STOCKS = [
    "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
    "HINDUNILVR.NS", "SBIN.NS", "BHARTIARTL.NS", "KOTAKBANK.NS", "ITC.NS",
    "LT.NS", "AXISBANK.NS", "ASIANPAINT.NS", "MARUTI.NS", "HCLTECH.NS",
    "SUNPHARMA.NS", "TITAN.NS", "BAJFINANCE.NS", "ULTRACEMCO.NS", "NTPC.NS",
    "WIPRO.NS", "NESTLEIND.NS", "POWERGRID.NS", "M&M.NS",
    "JSWSTEEL.NS", "ADANIENT.NS", "ADANIPORTS.NS", "TATASTEEL.NS", "ONGC.NS",
    "TECHM.NS", "HDFCLIFE.NS", "DIVISLAB.NS", "BAJAJFINSV.NS", "GRASIM.NS",
    "DRREDDY.NS", "CIPLA.NS", "BRITANNIA.NS", "EICHERMOT.NS", "APOLLOHOSP.NS",
    "COALINDIA.NS", "SBILIFE.NS", "BPCL.NS", "INDUSINDBK.NS", "TATACONSUM.NS",
    "HEROMOTOCO.NS", "HINDALCO.NS", "BAJAJ-AUTO.NS", "LTIM.NS", "SHRIRAMFIN.NS",
    "TRENT.NS", "POLYCAB.NS", "PERSISTENT.NS", "DIXON.NS", "TATAELXSI.NS",
    "ABB.NS", "SIEMENS.NS", "HAL.NS", "BEL.NS", "IRCTC.NS",
    "COFORGE.NS", "MUTHOOTFIN.NS", "INDHOTEL.NS", "BANKBARODA.NS",
    "PNB.NS", "IDFCFIRSTB.NS", "FEDERALBNK.NS", "CHOLAFIN.NS", "VEDL.NS",
]

def yf_download_safe(tickers, start, end, chunk_size=5, max_retries=5, base_sleep=2.0):
    out = {}
    total = len(tickers)
    print(f"📥 Downloading {total} stocks...")
    for i in range(0, total, chunk_size):
        chunk = tickers[i:i+chunk_size]
        for attempt in range(1, max_retries + 1):
            try:
                df = yf.download(" ".join(chunk), start=start, end=end,
                                group_by="ticker", auto_adjust=True,
                                threads=False, progress=False)
                if isinstance(df.columns, pd.MultiIndex):
                    for t in chunk:
                        if t in df.columns.get_level_values(0):
                            tdf = df[t].dropna(how="all")
                            if len(tdf) >= config.MIN_HISTORY_DAYS:
                                out[t] = tdf
                else:
                    if len(df) >= config.MIN_HISTORY_DAYS:
                        out[chunk[0]] = df.dropna(how="all")
                break
            except:
                time.sleep(base_sleep * (2 ** (attempt - 1)) + random.random())
        done = min(i + chunk_size, total)
        if done % 20 == 0 or done == total:
            print(f"   {done}/{total} | {len(out)} downloaded")
        time.sleep(base_sleep + random.random())
    print(f"✅ Downloaded {len(out)} stocks")
    return out

stock_data = yf_download_safe(FO_STOCKS, "2019-01-01", "2024-12-31")

# ============================================================

📥 Downloading 68 stocks...
   20/68 | 20 downloaded
   40/68 | 40 downloaded
   60/68 | 60 downloaded
   68/68 | 68 downloaded
✅ Downloaded 68 stocks


## CELL 5: Features V4 (Less aggressive capping)

In [6]:
# CELL 5: Features V4 (Less aggressive capping)
# ============================================================
def calculate_features_v4(df):
    data = df.copy()

    # Price Action (same as V2)
    data['return_1d'] = data['Close'].pct_change(1)
    data['return_5d'] = data['Close'].pct_change(5)
    data['return_10d'] = data['Close'].pct_change(10)
    data['return_20d'] = data['Close'].pct_change(20)
    data['volatility_20d'] = data['return_1d'].rolling(20).std() * np.sqrt(252)

    data['sma_20'] = data['Close'].rolling(20).mean()
    data['sma_50'] = data['Close'].rolling(50).mean()
    data['close_to_sma_20'] = (data['Close'] - data['sma_20']) / data['sma_20']
    data['close_to_sma_50'] = (data['Close'] - data['sma_50']) / data['sma_50']

    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    data['rsi_14_norm'] = (100 - (100 / (1 + rs)) - 50) / 50

    ema_12 = data['Close'].ewm(span=12).mean()
    ema_26 = data['Close'].ewm(span=26).mean()
    macd = ema_12 - ema_26
    macd_signal = macd.ewm(span=9).mean()
    data['macd_histogram_norm'] = (macd - macd_signal) / data['Close']

    bb_mid = data['Close'].rolling(20).mean()
    bb_std = data['Close'].rolling(20).std()
    bb_upper = bb_mid + 2 * bb_std
    bb_lower = bb_mid - 2 * bb_std
    data['bb_position'] = (data['Close'] - bb_lower) / (bb_upper - bb_lower + 1e-10)

    # SMC/ICT - V4: Use 7-day lookback (between V2's 10 and V3's 5)
    data['swing_high'] = data['High'].rolling(7, center=True).max()
    data['swing_low'] = data['Low'].rolling(7, center=True).min()
    data['prev_swing_high'] = data['swing_high'].shift(7)
    data['prev_swing_low'] = data['swing_low'].shift(7)
    data['higher_high'] = (data['swing_high'] > data['prev_swing_high']).astype(int)
    data['higher_low'] = (data['swing_low'] > data['prev_swing_low']).astype(int)
    data['lower_high'] = (data['swing_high'] < data['prev_swing_high']).astype(int)
    data['lower_low'] = (data['swing_low'] < data['prev_swing_low']).astype(int)

    data['structure_score'] = (data['higher_high'].rolling(5).sum() +
                               data['higher_low'].rolling(5).sum() -
                               data['lower_high'].rolling(5).sum() -
                               data['lower_low'].rolling(5).sum()) / 10

    # V4: Use 30-day range (between V2's 50 and V3's 20)
    range_high = data['High'].rolling(30).max()
    range_low = data['Low'].rolling(30).min()
    data['range_position'] = (data['Close'] - range_low) / (range_high - range_low + 1e-10)

    # V4: LESS AGGRESSIVE capping (±20% instead of ±15%)
    data['dist_to_swing_high'] = np.clip((data['swing_high'] - data['Close']) / data['Close'], -0.20, 0.20)
    data['dist_to_swing_low'] = np.clip((data['Close'] - data['swing_low']) / data['Close'], -0.20, 0.20)

    data['in_discount'] = (data['range_position'] < 0.5).astype(int)
    data['in_deep_discount'] = (data['range_position'] < 0.3).astype(int)
    data['in_premium'] = (data['range_position'] > 0.7).astype(int)

    vol_threshold = data['return_1d'].rolling(20).std() * 2
    data['near_bullish_ob'] = (data['return_1d'] > vol_threshold).rolling(10).sum()
    data['near_bearish_ob'] = (data['return_1d'] < -vol_threshold).rolling(10).sum()

    data['gap_up'] = ((data['Low'] > data['High'].shift(1)) & (data['return_1d'] > 0.01)).astype(int)
    data['gap_down'] = ((data['High'] < data['Low'].shift(1)) & (data['return_1d'] < -0.01)).astype(int)
    data['bullish_fvg'] = data['gap_up'].rolling(5).sum()
    data['bearish_fvg'] = data['gap_down'].rolling(5).sum()

    data['sweep_high'] = ((data['High'] > data['swing_high'].shift(1)) &
                          (data['Close'] < data['swing_high'].shift(1))).astype(int)
    data['sweep_low'] = ((data['Low'] < data['swing_low'].shift(1)) &
                         (data['Close'] > data['swing_low'].shift(1))).astype(int)

    data['bos_bullish'] = ((data['Close'] > data['swing_high'].shift(1)) &
                           (data['higher_high'] == 1)).astype(int)
    data['bos_bearish'] = ((data['Close'] < data['swing_low'].shift(1)) &
                           (data['lower_low'] == 1)).astype(int)

    # Volume - V4: Cap at 4x (between V2's uncapped and V3's 5x)
    data['volume_ma_20'] = data['Volume'].rolling(20).mean()
    data['volume_ratio'] = np.clip(data['Volume'] / (data['volume_ma_20'] + 1e-10), 0, 4)
    data['volume_trend'] = data['Volume'].rolling(5).mean() / (data['Volume'].rolling(20).mean() + 1e-10)

    obv = (np.sign(data['Close'].diff()) * data['Volume']).cumsum()
    data['obv_slope'] = obv.diff(5) / (obv.rolling(20).std() + 1e-10)

    data['vwap'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()
    data['close_to_vwap'] = (data['Close'] - data['vwap']) / (data['vwap'] + 1e-10)

    data['buying_pressure'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'] + 1e-10)
    data['accumulation_score'] = (data['buying_pressure'] * data['volume_ratio']).rolling(5).mean()
    data['big_volume_day'] = (data['volume_ratio'] > 2).astype(int)

    # Multi-timeframe (same as V2)
    data['daily_trend'] = (data['Close'] > data['sma_20']).astype(int)
    data['weekly_close'] = data['Close'].rolling(5).mean()
    data['weekly_high'] = data['High'].rolling(5).max()
    data['weekly_low'] = data['Low'].rolling(5).min()
    data['weekly_trend'] = (data['weekly_close'] > data['weekly_close'].shift(5)).astype(int)
    data['monthly_close'] = data['Close'].rolling(21).mean()
    data['monthly_trend'] = (data['monthly_close'] > data['monthly_close'].shift(21)).astype(int)
    data['mtf_alignment'] = (data['daily_trend'] + data['weekly_trend'] + data['monthly_trend']) / 3
    data['weekly_range_pos'] = (data['Close'] - data['weekly_low']) / (data['weekly_high'] - data['weekly_low'] + 1e-10)
    data['monthly_range_pos'] = (data['Close'] - data['Low'].rolling(21).min()) / \
                                (data['High'].rolling(21).max() - data['Low'].rolling(21).min() + 1e-10)
    data['trend_strength'] = abs(data['close_to_sma_20']) + abs(data['close_to_sma_50'])

    features_df = data[FEATURE_NAMES].copy()
    features_df = features_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return features_df

def create_labels_v4(df):
    forward_return = df['Close'].shift(-config.PREDICTION_HORIZON) / df['Close'] - 1
    labels = pd.Series(1, index=df.index)
    labels[forward_return >= config.UP_THRESHOLD] = 2
    labels[forward_return <= config.DOWN_THRESHOLD] = 0
    return labels

print("🔧 Calculating features...")
feature_data, labels_data = {}, {}
for symbol, data in stock_data.items():
    try:
        features = calculate_features_v4(data)
        labels = create_labels_v4(data)
        if len(features) > config.LOOKBACK_DAYS + config.PREDICTION_HORIZON:
            feature_data[symbol] = features
            labels_data[symbol] = labels
    except: pass
print(f"✅ Processed {len(feature_data)} stocks")

# ============================================================

🔧 Calculating features...
✅ Processed 68 stocks


## CELL 6: Dataset (Standard - no balanced sampling for CatBoost)

In [7]:
# CELL 6: Dataset (Standard - no balanced sampling for CatBoost)
# ============================================================
class SwingDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx): return {'features': self.features[idx], 'labels': self.labels[idx]}

class BalancedSwingDataset(Dataset):
    """Only for neural networks - with balanced sampling"""
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
        class_counts = Counter(labels)
        total = len(labels)
        self.class_weights = {c: total / (3 * count) for c, count in class_counts.items()}
        self.sample_weights = torch.FloatTensor([self.class_weights[int(l)] for l in labels])
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx): return {'features': self.features[idx], 'labels': self.labels[idx]}
    def get_sampler(self): return WeightedRandomSampler(self.sample_weights, len(self.sample_weights), True)

print("📊 Creating sequences...")
all_sequences, all_labels, all_dates = [], [], []
for symbol in feature_data.keys():
    features = feature_data[symbol].values
    labels = labels_data[symbol].values
    dates = feature_data[symbol].index
    for i in range(config.LOOKBACK_DAYS, len(features) - config.PREDICTION_HORIZON):
        seq = features[i-config.LOOKBACK_DAYS:i]
        label = labels[i]
        if not np.isnan(label) and not np.any(np.isnan(seq)):
            all_sequences.append(seq)
            all_labels.append(int(label))
            all_dates.append(dates[i])

sequences = np.array(all_sequences)
labels_arr = np.array(all_labels)
dates_arr = np.array(all_dates)

print(f"✅ Total sequences: {len(sequences):,}")

train_end = pd.Timestamp(config.TRAIN_END)
val_end = pd.Timestamp(config.VAL_END)
train_mask = dates_arr <= train_end
val_mask = (dates_arr > train_end) & (dates_arr <= val_end)
test_mask = dates_arr > val_end

# Standard dataset for CatBoost
train_dataset = SwingDataset(sequences[train_mask], labels_arr[train_mask])
val_dataset = SwingDataset(sequences[val_mask], labels_arr[val_mask])
test_dataset = SwingDataset(sequences[test_mask], labels_arr[test_mask])

# Balanced dataset for neural networks
train_balanced = BalancedSwingDataset(sequences[train_mask], labels_arr[train_mask])

print(f"📊 Train: {len(train_dataset):,} | Val: {len(val_dataset):,} | Test: {len(test_dataset):,}")
test_dist = Counter(labels_arr[test_mask])
print(f"📊 Test Distribution: SHORT={test_dist[0]}, NEUTRAL={test_dist[1]}, LONG={test_dist[2]}")

# ============================================================

📊 Creating sequences...
✅ Total sequences: 95,960
📊 Train: 87,800 | Val: 4,352 | Test: 3,808
📊 Test Distribution: SHORT=1493, NEUTRAL=1554, LONG=761


## CELL 7: Soft Focal Loss (reduced gamma)

In [8]:
# CELL 7: Soft Focal Loss (reduced gamma)
# ============================================================
class SoftFocalLoss(nn.Module):
    """Focal loss with reduced gamma for less aggressive reweighting"""
    def __init__(self, gamma=1.0, label_smoothing=0.05):
        super().__init__()
        self.gamma = gamma
        self.label_smoothing = label_smoothing

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', label_smoothing=self.label_smoothing)
        probs = torch.exp(-ce_loss)
        focal_weight = (1 - probs) ** self.gamma
        return (focal_weight * ce_loss).mean()

criterion = SoftFocalLoss(config.FOCAL_GAMMA, config.LABEL_SMOOTHING)
print(f"✅ Soft Focal Loss (gamma={config.FOCAL_GAMMA})")

# ============================================================

✅ Soft Focal Loss (gamma=1.0)


## CELL 8: CatBoost V4 - FIXED!

In [9]:
# CELL 8: CatBoost V4 - FIXED!
# ============================================================
print("="*60)
print("🚀 TRAINING CATBOOST V4")
print("="*60)

X_train = train_dataset.features[:, -1, :].numpy()
y_train = train_dataset.labels.numpy()
X_val = val_dataset.features[:, -1, :].numpy()
y_val = val_dataset.labels.numpy()

# Class weights (not too extreme)
class_counts = Counter(y_train)
total = len(y_train)
# Use sqrt to reduce extreme weights
class_weights = {c: np.sqrt(total / (3 * count)) for c, count in class_counts.items()}
print(f"Class weights (sqrt): {class_weights}")

catboost_model = CatBoostClassifier(
    iterations=config.CATBOOST_ITERATIONS,
    depth=config.CATBOOST_DEPTH,
    learning_rate=config.CATBOOST_LR,
    l2_leaf_reg=config.CATBOOST_L2_REG,
    loss_function='MultiClass',
    eval_metric='Accuracy',  # FIXED: Changed from TotalF1
    random_seed=42,
    verbose=100,
    early_stopping_rounds=100,  # FIXED: Increased from 50
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    class_weights=class_weights,
)

catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

cat_preds = catboost_model.predict(X_val)
cat_acc = (cat_preds == y_val).mean()
cat_f1 = f1_score(y_val, cat_preds, average='macro')
print(f"✅ CatBoost Val Accuracy: {cat_acc:.4f}, Macro-F1: {cat_f1:.4f}")

# Feature importance
importance = dict(zip(FEATURE_NAMES, catboost_model.feature_importances_))
sorted_imp = sorted(importance.items(), key=lambda x: x[1], reverse=True)[:10]
print("\n📊 Top 10 Features:")
for i, (f, v) in enumerate(sorted_imp):
    print(f"   {i+1}. {f}: {v:.2f}")

# ============================================================

# Save CatBoost model
try:
    catboost_model.save_model(os.path.join(RUN_DIR, 'catboost.cbm'))
    print('✅ Saved CatBoost model')
except Exception as e:
    print('⚠️ CatBoost save failed:', e)


🚀 TRAINING CATBOOST V4
Class weights (sqrt): {0: 1.0701389979000506, 1: 0.8927733299701714, 2: 1.0707886402611433}
0:	learn: 0.4662456	test: 0.4746418	best: 0.4746418 (0)	total: 156ms	remaining: 3m 53s
100:	learn: 0.4931090	test: 0.4853324	best: 0.4869139 (90)	total: 3.92s	remaining: 54.2s
200:	learn: 0.5054776	test: 0.4926612	best: 0.4935363 (199)	total: 4.83s	remaining: 31.2s
300:	learn: 0.5157998	test: 0.4942413	best: 0.4951579 (291)	total: 5.73s	remaining: 22.8s
400:	learn: 0.5241664	test: 0.4948638	best: 0.4977386 (350)	total: 6.63s	remaining: 18.2s
bestTest = 0.4977385503
bestIteration = 350
Shrink model to first 351 iterations.
✅ CatBoost Val Accuracy: 0.4071, Macro-F1: 0.4552

📊 Top 10 Features:
   1. dist_to_swing_high: 25.69
   2. dist_to_swing_low: 12.83
   3. weekly_range_pos: 11.31
   4. close_to_vwap: 7.26
   5. volatility_20d: 6.26
   6. macd_histogram_norm: 3.39
   7. volume_trend: 3.17
   8. structure_score: 2.63
   9. trend_strength: 2.59
   10. close_to_sma_50: 2.43


## CELL 9: TFT V4

In [10]:
# CELL 9: TFT V4
# ============================================================
class TFTModelV4(nn.Module):
    def __init__(self):
        super().__init__()
        h = config.TFT_HIDDEN_SIZE
        heads = config.TFT_ATTENTION_HEADS
        drop = config.TFT_DROPOUT

        self.var_sel = nn.Sequential(
            nn.Linear(40, h), nn.ReLU(), nn.Dropout(drop), nn.Linear(h, 40), nn.Softmax(dim=-1)
        )
        self.lstm = nn.LSTM(40, h, config.TFT_NUM_LAYERS, batch_first=True, dropout=drop if config.TFT_NUM_LAYERS > 1 else 0)
        self.attention = nn.MultiheadAttention(h, heads, dropout=drop, batch_first=True)
        self.norm = nn.LayerNorm(h)
        self.grn = nn.Sequential(nn.Linear(h, h), nn.GELU(), nn.Dropout(drop), nn.Linear(h, h))
        self.grn_gate = nn.Sequential(nn.Linear(h, h), nn.Sigmoid())
        self.grn_norm = nn.LayerNorm(h)
        self.output = nn.Linear(h, 3)

    def forward(self, x):
        w = self.var_sel(x.mean(1))
        x = x * w.unsqueeze(1)
        o, _ = self.lstm(x)
        a, _ = self.attention(o, o, o)
        o = self.norm(o + a)
        final = o[:, -1, :]
        g = self.grn_gate(final) * self.grn(final)
        out = self.grn_norm(final + g)
        return self.output(out)

print("="*60)
print("🚀 TRAINING TFT V4")
print("="*60)

tft_model = TFTModelV4().to(DEVICE)
# Use balanced sampler for neural networks
train_loader = DataLoader(train_balanced, batch_size=config.BATCH_SIZE, sampler=train_balanced.get_sampler(), num_workers=config.NUM_WORKERS, pin_memory=config.PIN_MEMORY, persistent_workers=(config.NUM_WORKERS>0))
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, num_workers=config.NUM_WORKERS, pin_memory=config.PIN_MEMORY, persistent_workers=(config.NUM_WORKERS>0))
optimizer = torch.optim.AdamW(tft_model.parameters(), lr=config.LEARNING_RATE, weight_decay=0.01)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

best_acc, patience_count, best_state = 0, 0, None
for epoch in range(config.MAX_EPOCHS):
    tft_model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = tft_model(batch['features'].to(DEVICE))
        loss = criterion(out, batch['labels'].to(DEVICE))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(tft_model.parameters(), 1.0)
        optimizer.step()
    scheduler.step()

    tft_model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for b in val_loader:
            preds.extend(tft_model(b['features'].to(DEVICE)).argmax(1).cpu().numpy())
            labels.extend(b['labels'].numpy())

    acc = np.mean(np.array(preds) == np.array(labels))
    f1 = f1_score(labels, preds, average='macro')
    print(f"Epoch {epoch+1}: Acc={acc:.4f}, F1={f1:.4f}")
    log_metric("TFT", "val", epoch+1, "acc", acc)
    log_metric("TFT", "val", epoch+1, "f1", f1)

    if acc > best_acc:
        best_acc, patience_count = acc, 0
        best_state = {k: v.cpu().clone() for k, v in tft_model.state_dict().items()}
        torch.save(best_state, os.path.join(RUN_DIR, 'tft_best.pt'))
    else:
        patience_count += 1
        if patience_count >= config.PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

tft_model.load_state_dict(best_state)
print(f"✅ TFT V4 Best Accuracy: {best_acc:.4f}")

# ============================================================

🚀 TRAINING TFT V4
Epoch 1: Acc=0.2863, F1=0.2500
Epoch 2: Acc=0.4239, F1=0.2903
Epoch 3: Acc=0.3387, F1=0.3207
Epoch 4: Acc=0.4644, F1=0.3355
Epoch 5: Acc=0.4653, F1=0.4517
Epoch 6: Acc=0.4216, F1=0.4266
Epoch 7: Acc=0.4274, F1=0.4308
Epoch 8: Acc=0.4573, F1=0.4500
Epoch 9: Acc=0.4708, F1=0.4597
Epoch 10: Acc=0.4841, F1=0.4528
Epoch 11: Acc=0.4871, F1=0.4569
Epoch 12: Acc=0.4304, F1=0.4335
Epoch 13: Acc=0.4688, F1=0.4638
Epoch 14: Acc=0.5198, F1=0.4596
Epoch 15: Acc=0.5000, F1=0.4862
Epoch 16: Acc=0.4989, F1=0.4725
Epoch 17: Acc=0.4922, F1=0.4666
Epoch 18: Acc=0.5211, F1=0.4812
Epoch 19: Acc=0.5011, F1=0.4937
Epoch 20: Acc=0.5191, F1=0.4963
Epoch 21: Acc=0.5221, F1=0.4858
Epoch 22: Acc=0.5124, F1=0.4958
Epoch 23: Acc=0.5018, F1=0.4869
Epoch 24: Acc=0.5110, F1=0.4818
Epoch 25: Acc=0.5087, F1=0.4874
Epoch 26: Acc=0.5145, F1=0.4933
Epoch 27: Acc=0.5039, F1=0.4807
Epoch 28: Acc=0.5071, F1=0.4877
Epoch 29: Acc=0.5101, F1=0.4891
Epoch 30: Acc=0.5078, F1=0.4885
Epoch 31: Acc=0.5349, F1=0.4714

## CELL 10: Stockformer V4

In [11]:
# CELL 10: Stockformer V4
# ============================================================
class StockformerV4(nn.Module):
    def __init__(self):
        super().__init__()
        d = config.STOCKFORMER_D_MODEL
        h = config.STOCKFORMER_N_HEADS
        n = config.STOCKFORMER_N_LAYERS
        drop = config.STOCKFORMER_DROPOUT

        self.trend_enc = nn.Sequential(nn.Linear(40, d), nn.LayerNorm(d), nn.GELU())
        self.seasonal_enc = nn.Sequential(nn.Linear(40, d), nn.LayerNorm(d), nn.GELU())
        self.residual_enc = nn.Sequential(nn.Linear(40, d), nn.LayerNorm(d), nn.GELU())

        self.pos = nn.Parameter(torch.randn(1, 60, d) * 0.02)

        self.trend_tf = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d, h, d*4, drop, batch_first=True), n
        )
        self.seasonal_tf = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d, h, d*4, drop, batch_first=True), n
        )
        self.residual_tf = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d, h, d*4, drop, batch_first=True), n
        )

        self.fusion = nn.Sequential(nn.Linear(d*3, d*2), nn.GELU(), nn.Linear(d*2, d))
        self.output = nn.Linear(d, 3)

    def forward(self, x):
        b, s, _ = x.shape
        trend = x.cumsum(1) / torch.arange(1, s+1, device=x.device).view(1, -1, 1)
        seasonal = x - trend

        t = self.trend_tf(self.trend_enc(trend) + self.pos[:, :s])[:, -1]
        se = self.seasonal_tf(self.seasonal_enc(seasonal) + self.pos[:, :s])[:, -1]
        r = self.residual_tf(self.residual_enc(x) + self.pos[:, :s])[:, -1]

        return self.output(self.fusion(torch.cat([t, se, r], -1)))

print("="*60)
print("🚀 TRAINING STOCKFORMER V4")
print("="*60)

sf_model = StockformerV4().to(DEVICE)
optimizer = torch.optim.AdamW(sf_model.parameters(), lr=config.LEARNING_RATE, weight_decay=0.01)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

best_acc, patience_count, best_state = 0, 0, None
for epoch in range(config.MAX_EPOCHS):
    sf_model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = sf_model(batch['features'].to(DEVICE))
        loss = criterion(out, batch['labels'].to(DEVICE))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(sf_model.parameters(), 1.0)
        optimizer.step()
    scheduler.step()

    sf_model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for b in val_loader:
            preds.extend(sf_model(b['features'].to(DEVICE)).argmax(1).cpu().numpy())
            labels.extend(b['labels'].numpy())

    acc = np.mean(np.array(preds) == np.array(labels))
    f1 = f1_score(labels, preds, average='macro')
    print(f"Epoch {epoch+1}: Acc={acc:.4f}, F1={f1:.4f}")
    log_metric("Stockformer", "val", epoch+1, "acc", acc)
    log_metric("Stockformer", "val", epoch+1, "f1", f1)

    if acc > best_acc:
        best_acc, patience_count = acc, 0
        best_state = {k: v.cpu().clone() for k, v in sf_model.state_dict().items()}
        torch.save(best_state, os.path.join(RUN_DIR, 'stockformer_best.pt'))
    else:
        patience_count += 1
        if patience_count >= config.PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

sf_model.load_state_dict(best_state)
print(f"✅ Stockformer V4 Best Accuracy: {best_acc:.4f}")

# ============================================================

🚀 TRAINING STOCKFORMER V4
Epoch 1: Acc=0.4660, F1=0.3086
Epoch 2: Acc=0.3656, F1=0.3466
Epoch 3: Acc=0.4076, F1=0.3978
Epoch 4: Acc=0.4219, F1=0.4138
Epoch 5: Acc=0.4320, F1=0.4194
Epoch 6: Acc=0.3966, F1=0.3936
Epoch 7: Acc=0.4007, F1=0.4010
Epoch 8: Acc=0.4412, F1=0.4228
Epoch 9: Acc=0.4336, F1=0.4164
Epoch 10: Acc=0.4246, F1=0.4114
Epoch 11: Acc=0.4258, F1=0.4118
Early stopping at epoch 11
✅ Stockformer V4 Best Accuracy: 0.4660


## CELL 11: Ensemble Evaluation

In [12]:
# CELL 11: Ensemble Evaluation
# ============================================================
print("="*60)
print("📊 ENSEMBLE EVALUATION V4")
print("="*60)

X_test = test_dataset.features.numpy()
y_test = test_dataset.labels.numpy()

cat_probs = catboost_model.predict_proba(X_test[:, -1, :])
tft_model.eval()
sf_model.eval()
with torch.no_grad():
    tft_probs = F.softmax(tft_model(torch.FloatTensor(X_test).to(DEVICE)), 1).cpu().numpy()
    sf_probs = F.softmax(sf_model(torch.FloatTensor(X_test).to(DEVICE)), 1).cpu().numpy()

# Static weighted ensemble (simpler, more stable)
ensemble_probs = (config.CATBOOST_WEIGHT * cat_probs +
                  config.TFT_WEIGHT * tft_probs +
                  config.STOCKFORMER_WEIGHT * sf_probs)

ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_conf = np.max(ensemble_probs, axis=1)

ensemble_acc = (ensemble_preds == y_test).mean()
ensemble_f1 = f1_score(y_test, ensemble_preds, average='macro')

print(f"\n🎯 ENSEMBLE TEST ACCURACY: {ensemble_acc:.4f} ({ensemble_acc*100:.1f}%)")
print(f"🎯 ENSEMBLE MACRO-F1: {ensemble_f1:.4f}")

print(f"\n📋 Individual Models:")
print(f"   CatBoost:    Acc={np.mean(cat_probs.argmax(1)==y_test):.4f}")
print(f"   TFT:         Acc={np.mean(tft_probs.argmax(1)==y_test):.4f}")
print(f"   Stockformer: Acc={np.mean(sf_probs.argmax(1)==y_test):.4f}")

print(f"\n📈 High-Confidence Performance:")
for thresh in [0.5, 0.6, 0.7, 0.8]:
    mask = ensemble_conf >= thresh
    if mask.sum() > 0:
        acc = (ensemble_preds[mask] == y_test[mask]).mean()
        print(f"   Conf >= {int(thresh*100)}%: {mask.sum():,} ({mask.mean()*100:.1f}%), Acc={acc:.4f}")

print(f"\n📊 Per-Class Recall:")
for c, name in [(0, 'SHORT'), (1, 'NEUTRAL'), (2, 'LONG')]:
    mask = y_test == c
    if mask.sum() > 0:
        recall = (ensemble_preds[mask] == c).mean()
        print(f"   {name}: {recall:.4f}")

# Model agreement
cat_preds = cat_probs.argmax(1)
tft_preds = tft_probs.argmax(1)
sf_preds = sf_probs.argmax(1)
agreement_3 = (cat_preds == tft_preds) & (tft_preds == sf_preds)
print(f"\n🤝 3/3 Agreement: {agreement_3.sum():,} ({agreement_3.mean()*100:.1f}%)")
if agreement_3.sum() > 0:
    print(f"   Accuracy when all agree: {(ensemble_preds[agreement_3] == y_test[agreement_3]).mean():.4f}")

print("\n" + "="*60)
print(classification_report(y_test, ensemble_preds, target_names=['SHORT', 'NEUTRAL', 'LONG']))

# ============================================================

📊 ENSEMBLE EVALUATION V4

🎯 ENSEMBLE TEST ACCURACY: 0.4803 (48.0%)
🎯 ENSEMBLE MACRO-F1: 0.4551

📋 Individual Models:
   CatBoost:    Acc=0.4727
   TFT:         Acc=0.4748
   Stockformer: Acc=0.3739

📈 High-Confidence Performance:
   Conf >= 50%: 410 (10.8%), Acc=0.7195
   Conf >= 60%: 86 (2.3%), Acc=0.8605
   Conf >= 70%: 9 (0.2%), Acc=0.8889

📊 Per-Class Recall:
   SHORT: 0.3208
   NEUTRAL: 0.6776
   LONG: 0.3903

🤝 3/3 Agreement: 1,688 (44.3%)
   Accuracy when all agree: 0.4733

              precision    recall  f1-score   support

       SHORT       0.64      0.32      0.43      1493
     NEUTRAL       0.46      0.68      0.55      1554
        LONG       0.39      0.39      0.39       761

    accuracy                           0.48      3808
   macro avg       0.50      0.46      0.46      3808
weighted avg       0.52      0.48      0.47      3808



## CELL 12: Save Models

In [13]:
# CELL 12: Save Models
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

os.makedirs(config.MODEL_SAVE_PATH, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("💾 Saving V4 models...")
catboost_model.save_model(f"{config.MODEL_SAVE_PATH}catboost_v4.cbm")
torch.save(tft_model.state_dict(), f"{config.MODEL_SAVE_PATH}tft_v4.pt")
torch.save(sf_model.state_dict(), f"{config.MODEL_SAVE_PATH}stockformer_v4.pt")

model_config = {
    "version": "V4_Fixed",
    "feature_columns": FEATURE_NAMES,
    "thresholds": {"up": config.UP_THRESHOLD, "down": config.DOWN_THRESHOLD},
    "ensemble_weights": {"catboost": config.CATBOOST_WEIGHT, "tft": config.TFT_WEIGHT, "stockformer": config.STOCKFORMER_WEIGHT},
    "test_accuracy": float(ensemble_acc),
    "test_macro_f1": float(ensemble_f1),
    "trained_at": timestamp
}
with open(f"{config.MODEL_SAVE_PATH}model_config_v4.json", "w") as f:
    json.dump(model_config, f, indent=2)

print(f"✅ V4 Models saved to: {config.MODEL_SAVE_PATH}")
print(f"🎉 Training complete! Accuracy: {ensemble_acc*100:.1f}%, Macro-F1: {ensemble_f1:.4f}")

# Save run summary
with open(os.path.join(RUN_DIR, 'run_summary.json'), 'w') as f:
    json.dump({
        'run_id': RUN_ID,
        'run_dir': RUN_DIR,
        'created_at': datetime.now().isoformat(timespec='seconds'),
    }, f, indent=2)

WRITER.flush(); WRITER.close()
print('✅ Tracking closed. To view TensorBoard in Colab:')
print('   %load_ext tensorboard')
print('   %tensorboard --logdir', os.path.join(RUN_DIR, 'tb'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
💾 Saving V4 models...
✅ V4 Models saved to: /content/drive/MyDrive/SwingAI/models_v4/
🎉 Training complete! Accuracy: 48.0%, Macro-F1: 0.4551
✅ Tracking closed. To view TensorBoard in Colab:
   %load_ext tensorboard
   %tensorboard --logdir /content/drive/MyDrive/SwingAI_runs/run_20260107_182456/tb
